# コンペティション課題2

## 課題

CNNを用いて、[Tensorflow Flower Dataset](https://www.tensorflow.org/tutorials/load_data/images)のクラス分類を行います。

## 目標値

Accuracy: 80%  
（これはあくまで「目標値」であるため、達成できなかったからといって不合格となったり、著しく成績が損なわれることはありません）

## ルール

- 「修正しないでください」とあるセルを、修正しないでください。
- モデルのアーキテクチャは自由です。講義で扱ったモデル以外でも構いません。
- 訓練データはx_train, t_train、テストデータはx_testで与えられます。
- 予測ラベルは one_hot表現ではなく0~9のクラスラベルによって表してください。
- 以下のセルで定義されているx_train, t_train以外の学習データは使わないでください。

## 提出方法
- 1つのファイルを提出していただきます。
  1. テストデータ`test_iter`に対する予測ラベルを`submission1_pred.csv`として保存・ダウンロードし、Homeworkタブから**Day1 Pred (.csv)**を選択して提出してください。
  2. それとは別に、最終提出に対応するノートブックをわかるようにiLect上に置いておいてください。


- 成績優秀者には、次回講義にて取り組みの発表をお願いいたします。

## 評価方法

- 予測ラベルのt_testに対する精度 (Accuracy) で評価します。

## LeaderBoard

- コンペティション期間中のLeaderBoardは提出されたcsvファイルのうち50%を使って計算されます。コンペティション終了時には提出されたcsvファイルのうち、コンペティション期間中のLeaderBoard計算に使われなかったもう半分のデータがスコア計算に使用されます。
- このため、コンペ中の順位とコンペ終了後にLeaderBoardが更新された後の順位やスコアが食い違うことがあります。

## データの読み込み (仮のコード・そもそもtestデータを作っていない）

- このセルは修正しないでください。
- 誤って修正した場合は、元ファイルをコピーし直してください。

In [1]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import torch.autograd as autograd
import torch.nn.functional as F
from torchvision import transforms, datasets, models
import matplotlib.pyplot as plt

In [2]:
class train_dataset(torch.utils.data.Dataset):
    def __init__(self, x_train, t_train, transform):
        self.x_train = x_train
        self.t_train = t_train.astype('int32')
        self.transform = transform

    def __len__(self):
        return self.x_train.shape[0]

    def __getitem__(self, idx):
        data = torch.tensor(self.x_train[idx], dtype=torch.float)
        data = torch.transpose(data,0,2)
        data = torch.transpose(data,1,2)
        label = torch.tensor(self.t_train[idx], dtype=torch.long)
        if self.transform:
            data = self.transform(data)
        return data, label


class test_dataset(torch.utils.data.Dataset):
    def __init__(self, x_test, transform):
        self.x_test = x_test
        self.transform = transform

    def __len__(self):
        return self.x_test.shape[0]

    def __getitem__(self, idx):
        data = torch.tensor(self.x_test[idx], dtype=torch.float)
        data = torch.transpose(data,0,2)
        data = torch.transpose(data,1,2)
        if self.transform:
            data = self.transform(data)
        return data
    




### 畳み込みニューラルネットワーク(CNN)の実装

In [3]:
# %%writefile /root/userspace/submission2_code.py
# 上記のコメントアウトを外し'%%'以下を実行すると、提出コードが出力できます


# 精度向上ポイント: Data Augmentation手法
transform_train =  transforms.Compose([])
transform_test = transforms.Compose([])

# data augmentationとデータ整形
height = 224
width =224
batch_size = 1
transform_train = transforms.Compose(
    [
        transforms.ToPILImage(),
        transforms.Resize((height,width)),   
        transforms.RandomHorizontalFlip(),
        transforms.RandomRotation(10),
        transforms.RandomAffine(translate=(0.05,0.05), degrees=0),
         # 画像によってサイズが変わってはならない。
#          transforms.CenterCrop(192),
        transforms.ToTensor(),      
    ]
)
transform_test = transforms.Compose(
[
    transforms.ToPILImage(),
    transforms.Resize((height,width)),
    transforms.ToTensor()
]
)
x_train = np.load('/root/userspace/public/day2/homework2/data/x_train.npy')
t_train = np.load('/root/userspace/public/day2/homework2/data/t_train.npy')
    
# テストデータ
x_test = np.load('/root/userspace/public/day2/homework2/data/x_test.npy')

trainval_data = train_dataset(x_train, t_train, transform_train)
test_data = test_dataset(x_test,transform_test)
val_ratio = 0.1
val_size = int(len(trainval_data) * val_ratio)
train_size = len(trainval_data) - val_size

train_data, val_data = torch.utils.data.random_split(trainval_data, [train_size, val_size])

# 精度向上ポイント: バッチサイズ
batch_size = 32

dataloader_train = torch.utils.data.DataLoader(
    train_data,
    batch_size=batch_size,
    shuffle=True
)

dataloader_valid = torch.utils.data.DataLoader(
    val_data,
    batch_size=batch_size,
    shuffle=True
)

dataloader_test = torch.utils.data.DataLoader(
    test_data,
    batch_size=batch_size,
    shuffle=False
)

rng = np.random.RandomState(1234)
random_state = 42
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [4]:
class GAP(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self,x):
        return F.adaptive_avg_pool2d(x,(1,1)).squeeze()

# def init_weights(m):  # Heの初期化
#     if type(m) == nn.Linear or type(m) == nn.Conv2d:
#         torch.nn.init.kaiming_normal_(m.weight)
#         m.bias.data.fill_(0.0)
def init_weights(m):  # xavierの初期化
    if type(m) == nn.Linear or type(m) == nn.Conv2d:
        torch.nn.init.xavier_uniform_(m.weight)
        m.bias.data.fill_(0.0)

In [5]:
model_tuned = models.resnet152(pretrained = True)
model_tuned.avgpool = GAP() # dense layerをGAPに変更
for name, child in model_tuned.named_children():
    print(name)
    
# for param in model_tuned.parameters(): # pretrained部分の重みを固定する
#     param.requires_grad = False
    
for name, child in model_tuned.named_children():
    if name in ['layer4','avgpool','fc']:
#     if name in ['layer3','layer4']:    
        print(name + ' is unfrozen')
        for param in child.parameters():
            param.requires_grad = True
    else:
        print(name + ' is frozen')
        for param in child.parameters():
            param.requires_grad = False

in_dim = 2048 #入力次元数
out_dim = 5 # 出力クラス数
model_tuned.fc = nn.Sequential(
#      nn.BatchNorm1d(in_dim),
#      nn.Dropout(0.2,inplace=False),
#      nn.Linear(in_dim,520),
#      nn.ReLU(True),
#      nn.Linear(520,out_dim)
    nn.BatchNorm1d(in_dim),
    nn.Linear(in_dim,1000), # classifierの書き換え
    nn.ReLU(True),
    nn.Dropout(0.2,inplace=False),
    nn.Linear(1000,512),
    nn.ReLU(True),
    nn.Dropout(0.2,inplace=False),
    nn.Linear(512,out_dim)
)

model_tuned.fc.apply(init_weights) # 重み・バイアスの初期化を行う
model_tuned.to(device)

Downloading: "https://download.pytorch.org/models/resnet152-b121ed2d.pth" to /root/.cache/torch/checkpoints/resnet152-b121ed2d.pth



conv1
bn1
relu
maxpool
layer1
layer2
layer3
layer4
avgpool
fc
conv1 is frozen
bn1 is frozen
relu is frozen
maxpool is frozen
layer1 is frozen
layer2 is frozen
layer3 is frozen
layer4 is unfrozen
avgpool is unfrozen
fc is unfrozen


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [6]:
lr = 0.01
optimizer_tuned = optim.Adam(filter(lambda p: p.requires_grad,model_tuned.parameters()), lr=lr,weight_decay=1e-4)
# optimizer_tuned optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer_tuned,step_size=3,gamma=0.4)

loss_function = nn.CrossEntropyLoss()
n_epochs =20
for epoch in range(n_epochs):
    losses_train = []
    losses_valid = []

    model_tuned.train()
    n_train = 0
    acc_train = 0
    for x, t in dataloader_train:
        n_train += t.size()[0]
        model_tuned.zero_grad()  # 勾配の初期化
        
        x = x.to(device)  # テンソルをGPUに移動
        t = t.to(device)
       
        y = model_tuned.forward(x)  # 順伝播

        loss = loss_function(y, t)  # 誤差(クロスエントロピー誤差関数)の計算
#         l1_lambda = 0.00001
#         l1_norm = sum(p.abs().sum() for p in model_tuned.parameters())
#         loss = loss + l1_lambda * l1_norm     
#         optimizer_tuned.zero_grad()  
        loss.backward()  # 誤差の逆伝播

        optimizer_tuned.step()  # パラメータの更新

        pred = y.argmax(1)  # 最大値を取るラベルを予測ラベルとする
        
        acc_train += (pred == t).float().sum().item()
        losses_train.append(loss.tolist())
    scheduler.step()
    model_tuned.eval()
    n_val = 0
    acc_val = 0
    for x, t in dataloader_valid:
        n_val += t.size()[0]

        x = x.to(device)  # テンソルをGPUに移動
        t = t.to(device)
        y = model_tuned.forward(x)  # 順伝播

        loss = loss_function(y, t)  # 誤差(クロスエントロピー誤差関数)の計算

        pred = y.argmax(1)  # 最大値を取るラベルを予測ラベルとする
        acc_val += (pred == t).float().sum().item()
        losses_valid.append(loss.tolist())
    print('\n learning rate ',optimizer_tuned.param_groups[0]['lr'])
    print('EPOCH: {}, Train [Loss: {:.3f}, Accuracy: {:.3f}], Valid [Loss: {:.3f}, Accuracy: {:.3f}]'.format(
        epoch+1,
        np.mean(losses_train),
        acc_train/n_train,
        np.mean(losses_valid),
        acc_val/n_val
    ))


 learning rate  0.01
EPOCH: 1, Train [Loss: 4.218, Accuracy: 0.598], Valid [Loss: 0.789, Accuracy: 0.755]

 learning rate  0.01
EPOCH: 2, Train [Loss: 0.911, Accuracy: 0.746], Valid [Loss: 0.512, Accuracy: 0.847]

 learning rate  0.004
EPOCH: 3, Train [Loss: 0.749, Accuracy: 0.762], Valid [Loss: 0.574, Accuracy: 0.795]

 learning rate  0.004
EPOCH: 4, Train [Loss: 0.490, Accuracy: 0.849], Valid [Loss: 0.301, Accuracy: 0.890]

 learning rate  0.004
EPOCH: 5, Train [Loss: 0.392, Accuracy: 0.871], Valid [Loss: 0.330, Accuracy: 0.876]

 learning rate  0.0016
EPOCH: 6, Train [Loss: 0.434, Accuracy: 0.876], Valid [Loss: 0.412, Accuracy: 0.862]

 learning rate  0.0016
EPOCH: 7, Train [Loss: 0.275, Accuracy: 0.912], Valid [Loss: 0.282, Accuracy: 0.908]

 learning rate  0.0016
EPOCH: 8, Train [Loss: 0.277, Accuracy: 0.915], Valid [Loss: 0.357, Accuracy: 0.899]

 learning rate  0.00064
EPOCH: 9, Train [Loss: 0.238, Accuracy: 0.924], Valid [Loss: 0.324, Accuracy: 0.890]

 learning rate  0.00064


In [8]:
for name, child in model_tuned.named_children():
    if name in ['layer2','layer3']:
        print(name + ' has been unfrozen.')
        for param in child.parameters():
            param.requires_grad = True
    else:
        print(name + ' is frozen')
        for param in child.parameters():
            param.requires_grad = False

lr=0.0003
optimizer_tuned = optim.Adagrad(filter(lambda p: p.requires_grad,model_tuned.parameters()), lr=lr)
# optimizer_tuned optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer_tuned,step_size=4,gamma=0.1)

loss_function = nn.CrossEntropyLoss()
n_epochs =20
for epoch in range(n_epochs):
    losses_train = []
    losses_valid = []

    model_tuned.train()
    n_train = 0
    acc_train = 0
    for x, t in dataloader_train:
        n_train += t.size()[0]
        model_tuned.zero_grad()  # 勾配の初期化
        
        x = x.to(device)  # テンソルをGPUに移動
        t = t.to(device)
       
        y = model_tuned.forward(x)  # 順伝播

        loss = loss_function(y, t)  # 誤差(クロスエントロピー誤差関数)の計算
#         l1_lambda = 0.00001
#         l1_norm = sum(p.abs().sum() for p in model_tuned.parameters())
#         loss = loss + l1_lambda * l1_norm     
#         optimizer_tuned.zero_grad()  
        loss.backward()  # 誤差の逆伝播

        optimizer_tuned.step()  # パラメータの更新

        pred = y.argmax(1)  # 最大値を取るラベルを予測ラベルとする
        
        acc_train += (pred == t).float().sum().item()
        losses_train.append(loss.tolist())
    scheduler.step()
    model_tuned.eval()
    n_val = 0
    acc_val = 0
    for x, t in dataloader_valid:
        n_val += t.size()[0]

        x = x.to(device)  # テンソルをGPUに移動
        t = t.to(device)
        y = model_tuned.forward(x)  # 順伝播

        loss = loss_function(y, t)  # 誤差(クロスエントロピー誤差関数)の計算

        pred = y.argmax(1)  # 最大値を取るラベルを予測ラベルとする
        acc_val += (pred == t).float().sum().item()
        losses_valid.append(loss.tolist())
    print('\n learning rate ',optimizer_tuned.param_groups[0]['lr'])
    print('EPOCH: {}, Train [Loss: {:.3f}, Accuracy: {:.3f}], Valid [Loss: {:.3f}, Accuracy: {:.3f}]'.format(
        epoch+1,
        np.mean(losses_train),
        acc_train/n_train,
        np.mean(losses_valid),
        acc_val/n_val
    ))

conv1 is frozen
bn1 is frozen
relu is frozen
maxpool is frozen
layer1 is frozen
layer2 has been unfrozen.
layer3 has been unfrozen.
layer4 is frozen
avgpool is frozen
fc is frozen

 learning rate  0.0003
EPOCH: 1, Train [Loss: 0.278, Accuracy: 0.921], Valid [Loss: 0.217, Accuracy: 0.919]

 learning rate  0.0003
EPOCH: 2, Train [Loss: 0.128, Accuracy: 0.960], Valid [Loss: 0.257, Accuracy: 0.916]

 learning rate  0.0003
EPOCH: 3, Train [Loss: 0.095, Accuracy: 0.973], Valid [Loss: 0.221, Accuracy: 0.934]

 learning rate  2.9999999999999997e-05
EPOCH: 4, Train [Loss: 0.092, Accuracy: 0.977], Valid [Loss: 0.243, Accuracy: 0.931]

 learning rate  2.9999999999999997e-05
EPOCH: 5, Train [Loss: 0.070, Accuracy: 0.980], Valid [Loss: 0.195, Accuracy: 0.934]

 learning rate  2.9999999999999997e-05
EPOCH: 6, Train [Loss: 0.067, Accuracy: 0.982], Valid [Loss: 0.214, Accuracy: 0.925]

 learning rate  2.9999999999999997e-05
EPOCH: 7, Train [Loss: 0.060, Accuracy: 0.985], Valid [Loss: 0.193, Accuracy: 

In [13]:
model_tuned.to(device)
model_tuned.eval()
t_pred = []
for x in dataloader_test:
    x = x.to(device)
    output = model_tuned(x)
    pred = output.argmax(1).tolist()
    t_pred.extend(pred)

submission = pd.Series(t_pred, name='label')
submission.to_csv('/root/userspace/submission3_final.csv', header=True, index_label='id')

RuntimeError: CUDA out of memory. Tried to allocate 20.00 MiB (GPU 0; 11.17 GiB total capacity; 10.35 GiB already allocated; 3.44 MiB free; 10.86 GiB reserved in total by PyTorch)

In [ ]:
for name, child in model_tuned.named_children():
    if name in ['layer1']:
        print(name + ' has been unfrozen.')
        for param in child.parameters():
            param.requires_grad = True
    else:
        print(name + ' is frozen')
        for param in child.parameters():
            param.requires_grad = False

lr=0.0003
optimizer_tuned = optim.Adagrad(filter(lambda p: p.requires_grad,model_tuned.parameters()), lr=lr)
# optimizer_tuned optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer_tuned,step_size=4,gamma=0.1)

loss_function = nn.CrossEntropyLoss()
n_epochs =20
for epoch in range(n_epochs):
    losses_train = []
    losses_valid = []

    model_tuned.train()
    n_train = 0
    acc_train = 0
    for x, t in dataloader_train:
        n_train += t.size()[0]
        model_tuned.zero_grad()  # 勾配の初期化
        
        x = x.to(device)  # テンソルをGPUに移動
        t = t.to(device)
       
        y = model_tuned.forward(x)  # 順伝播

        loss = loss_function(y, t)  # 誤差(クロスエントロピー誤差関数)の計算
#         l1_lambda = 0.00001
#         l1_norm = sum(p.abs().sum() for p in model_tuned.parameters())
#         loss = loss + l1_lambda * l1_norm     
#         optimizer_tuned.zero_grad()  
        loss.backward()  # 誤差の逆伝播

        optimizer_tuned.step()  # パラメータの更新

        pred = y.argmax(1)  # 最大値を取るラベルを予測ラベルとする
        
        acc_train += (pred == t).float().sum().item()
        losses_train.append(loss.tolist())
    scheduler.step()
    model_tuned.eval()
    n_val = 0
    acc_val = 0
    for x, t in dataloader_valid:
        n_val += t.size()[0]

        x = x.to(device)  # テンソルをGPUに移動
        t = t.to(device)
        y = model_tuned.forward(x)  # 順伝播

        loss = loss_function(y, t)  # 誤差(クロスエントロピー誤差関数)の計算

        pred = y.argmax(1)  # 最大値を取るラベルを予測ラベルとする
        acc_val += (pred == t).float().sum().item()
        losses_valid.append(loss.tolist())
    print('\n learning rate ',optimizer_tuned.param_groups[0]['lr'])
    print('EPOCH: {}, Train [Loss: {:.3f}, Accuracy: {:.3f}], Valid [Loss: {:.3f}, Accuracy: {:.3f}]'.format(
        epoch+1,
        np.mean(losses_train),
        acc_train/n_train,
        np.mean(losses_valid),
        acc_val/n_val
    ))